In [152]:
# !export PATH="${PATH}:/opt/gurobi912/linux64/bin"
# !export GUROBI_HOME="/home/victor-duraes/opt/gurobi912/linux64"
# !export LD_LIBRARY_PATH="${LD_LIBRARY_PATH}:${GUROBI_HOME}/lib"

#Portfolio optimization all three energy sources

from __future__ import division
from pyomo.environ import *
import numpy as np
import time
from GetIdxInRadious import GetIdxOutRadious


start_time = time.time()
S_CPU_TIME=time.perf_counter()
######################

#LCOE range we want to investigate
#LCOE_Max=range(500,50,-10)

LCOE_Max=range(500,50,-2)
Nt=[0,0,150]# Total number of installed turbine units Wind/Wave/Ocean
#Upper bound for the number of installed turbines in a single cell Wind/Wave/Ocean
NuWiWaOcean=[4,50,4]
Radious=40
alpha=0.9

ResultsFileName='PortfolioOptimizationWindWaveOcean'+str(Nt[0])+"_"+str(Nt[1])+"_"+str(Nt[2])


In [153]:
Data=np.load("/home/victor-duraes/Projects/Ocean2/Mean_Cvar/SyntheticData_CVAR_S.npz")
#Wind
WindEnergy=Data["WindEnergy"] #[pu]
WindLatLong=Data["WindLatLong"]
AnnualizedCostWind=Data["AverageAnnualCostPerWindTurbine"]*10**6 #[$/Year]
NumWindSites=WindEnergy.shape[1]
MaxNumOfWindTurbines=Data["MaxNumOfWindTurbines"]
RatedPowerWind=Data["RatedPowerWind"]
MaxNumOfWindTurbines[MaxNumOfWindTurbines>100]=100

#Wave
WaveEnergy=Data["WaveEnergy"] #[pu]
WaveLatLong=Data["WaveLatLong"]
NumWaveSites=WaveEnergy.shape[1]
RatedPowerWave=Data["RatedPowerWave"]
AnnualizedCostWave=Data["AverageAnnualCostPerWaveTurbine"]*10**6 #[$/Year-unit]
MaxNumOfWaveTurbines=Data["MaxNumOfWaveTurbines"]
MaxNumOfWaveTurbines[MaxNumOfWaveTurbines>100]=100

#Ocean
OceanEnergy=Data["OceanEnergy"] #[pu]
OceanLatLong=Data["OceanLatLong"]
AnnualizedCostOcean=Data["AverageAnnualCostPerOceanTurbine"]*10**6 #[$/Year]
NumOceanSites=OceanEnergy.shape[1]      
RatedPowerOcean=Data["RatedPowerOcean"]
MaxNumOfOceanTurbines=Data["MaxNumOfOceanTurbines"]
MaxNumOfOceanTurbines[MaxNumOfOceanTurbines>100]=100

RatedPowerSystem=(Nt[0]*RatedPowerWind+Nt[1]*RatedPowerWave+Nt[2]*RatedPowerOcean)*10**(-6)

#Vectorize maximum number of turbines per site location per technology
Nu=np.concatenate((MaxNumOfWindTurbines,MaxNumOfWaveTurbines,MaxNumOfOceanTurbines))

#Vectorize total number of installed turbines per technology
RHS_S=np.concatenate((np.full((NumWindSites), Nt[0]),np.full((NumWaveSites), Nt[1]),np.full((NumOceanSites), Nt[2])))

#Vectorize annualized cost for each site location and per technology
AnnCost=np.concatenate((AnnualizedCostWind,AnnualizedCostWave,AnnualizedCostOcean)) #Annualized cost [$/Year]

#Vectorize energy generation in each site location and energy resource
EnergyGeneration=np.concatenate((WindEnergy,WaveEnergy,OceanEnergy),axis=1)
EnergyGeneration_AVG=np.average(EnergyGeneration,axis=0)*8766#[MWh/Year] (8766 is the number of hours in a year)
NumSites=NumWindSites+NumWaveSites+NumOceanSites
NScenarios=OceanEnergy.shape[0]

In [154]:

MINLP = ConcreteModel()
MINLP.SiteWind = RangeSet(0,NumWindSites-1)
MINLP.SiteWave = RangeSet(NumWindSites,NumWindSites+NumWaveSites-1)
MINLP.SiteOcean = RangeSet(NumWindSites+NumWaveSites,NumSites-1)
MINLP.Site = RangeSet(0,NumSites-1)

MINLP.y = Var(MINLP.Site, domain=NonNegativeIntegers)# Integer variable to track the number of turbines used per site location
MINLP.s = Var(MINLP.Site, domain=Binary)# Integer variable to track the site location used for each technology
MINLP.c = Var( domain=NonPositiveReals)
MINLP.z = Var(range(NScenarios),domain=NonNegativeReals)

def objective_rule(MINLP):   
    CVaR=MINLP.c+1/((1-alpha)*NScenarios)*sum(MINLP.z[i] for i in range(NScenarios))
    return CVaR

def CVaRZ(MINLP,i):
    return MINLP.z[i]>=summation(-EnergyGeneration[i,:], MINLP.y)-MINLP.c

MINLP.CVaRZ = Constraint(range(NScenarios), rule=CVaRZ)

MINLP.OBJ = Objective(rule = objective_rule, sense=minimize)

def NumTurbinesCell_rule(MINLP,i):
    return MINLP.y[i]<=Nu[i]

MINLP.Turbines_Cell = Constraint(MINLP.Site, rule=NumTurbinesCell_rule)

MINLP.TMaxTurbinesWind = Constraint(expr=sum(MINLP.y[i] for i in MINLP.SiteWind)==Nt[0])
MINLP.ChooseOneCircleWind1= Constraint(expr=sum(MINLP.s[i] for i in MINLP.SiteWind)==1)
    
MINLP.TMaxTurbinesWave = Constraint(expr=sum(MINLP.y[i] for i in MINLP.SiteWave)==Nt[1])
MINLP.ChooseOneCircleWave1= Constraint(expr=sum(MINLP.s[i] for i in MINLP.SiteWave)==1)

MINLP.TMaxTurbinesOcean = Constraint(expr=sum(MINLP.y[i] for i in MINLP.SiteOcean)==Nt[2])
MINLP.ChooseOneCircleOcean1= Constraint(expr=sum(MINLP.s[i] for i in MINLP.SiteOcean)==1)
    

IdxOut=GetIdxOutRadious(Radious, WindLatLong, WaveLatLong, OceanLatLong)

def MaximumRadious(MINLP,i):  
    
    return sum(MINLP.y[j] for j in np.where(IdxOut[i,:])[0])<=(1-MINLP.s[i])*RHS_S[i]        

MINLP.Maximum_Radious = Constraint(MINLP.Site, rule=MaximumRadious)

opt = SolverFactory('gurobi', solver_io="python")
opt.options['mipgap'] = 0.01
#opt.options['max_iter'] = 500


In [155]:
SaveFeasibility, Save_LCOETarget, Save_S, Save_LCOE_Achieved = list(), list(), list(), list()
SaveYWind, SaveYWave, SaveYOcean,SaveCVaR = list(), list(), list(), list()
SaveYWind_LatLong, SaveYWave_LatLong, SaveYOcean_LatLong = list(), list(), list()

LowestLCOE=10**10
for LCOE_Idx in range(len(LCOE_Max)):
    LCOETarget=LCOE_Max[LCOE_Idx]
    
    if LCOETarget<LowestLCOE:    
        Bypass=0
        #upperbound for the LCOE
        MINLP.LCOE_Target = Constraint(expr=(sum(MINLP.y[i]*AnnCost[i] for i in MINLP.Site)-LCOETarget*sum(MINLP.y[i]*EnergyGeneration_AVG[i] for i in MINLP.Site)) <= 0)
       
        print("Running Model With LCOE= %.2f" % LCOETarget)
        
        try:
            results=opt.solve(MINLP, tee=True)
        except:
            Bypass=1
            MINLP.del_component(MINLP.LCOE_Target)  
    
        if Bypass==0:
            if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
                SaveFeasibility.append(1)
                Save_LCOETarget.append(LCOETarget)
                
                Optimal_Y=MINLP.y.get_values()
                Optimal_S=MINLP.s.get_values()
                
                Optimal_Y=np.reshape(np.array([Optimal_Y[i] for i in MINLP.Site]),(1,NumSites))
                Optimal_S=np.reshape(np.array([Optimal_S[i] for i in MINLP.Site]),(1,NumSites))
                
                Save_S.append(Optimal_S)
                
        
                CurrentLCOE=(np.sum(Optimal_Y*AnnCost)/np.sum(Optimal_Y*EnergyGeneration_AVG))
                Save_LCOE_Achieved.append(CurrentLCOE)
                SaveCVaR.append(-value(MINLP.OBJ))
                if LowestLCOE>CurrentLCOE:
                    LowestLCOE=CurrentLCOE
                    
                if Nt[0]!=0:
                    TempXWind=Optimal_Y[0, MINLP.SiteWind]
                    IdxWind=np.reshape(np.array([TempXWind> 10**-2]),-1)
                    SaveYWind.append(TempXWind[IdxWind])
                    SaveYWind_LatLong.append(WindLatLong[IdxWind,:])  
                    
                if Nt[1]!=0:  
                    TempXWave=Optimal_Y[0, MINLP.SiteWave]
                    IdxWave=np.reshape(np.array([TempXWave> 10**-2]),-1)
                    SaveYWave.append(TempXWave[IdxWave])
                    SaveYWave_LatLong.append(WaveLatLong[IdxWave,:])
        
                if Nt[2]!=0:        
                    TempXOcean=Optimal_Y[0, MINLP.SiteOcean]
                    IdxOcean=np.reshape(np.array([TempXOcean> 10**-2]),-1)
                    SaveYOcean.append(TempXOcean[IdxOcean])
                    SaveYOcean_LatLong.append(OceanLatLong[IdxOcean,:])
                
                #Delete constraint for its modification in the next step of the for loop
                MINLP.del_component(MINLP.LCOE_Target)
            
            else:# Something else is wrong
                MINLP.del_component(MINLP.LCOE_Target)   
                SaveFeasibility.append(0)
                SaveCVaR.append(None)
                Save_LCOETarget.append(None)
                Save_S.append(None)
                Save_LCOE_Achieved.append(None)
                SaveYWind.append(None)                   
                SaveYWave.append(None)
                SaveYOcean.append(None)               
                SaveYWind_LatLong.append(None)
                SaveYWave_LatLong.append(None)                
                SaveYOcean_LatLong.append(None)
                break


Running Model With LCOE= 500.00
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter LogFile to /tmp/tmpz1tyoxmo.log
   Prev:   Default: 
Changed value of parameter mipgap to 0.01
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 64 physical cores, 128 logical processors, using up to 32 threads
Optimize a model with 10711 rows, 10705 columns and 3578867 nonzeros
Model fingerprint: 0x69c10ade
Variable types: 10001 continuous, 704 integer (352 binary)
Coefficient statistics:
  Matrix range     [1e-07, 8e+06]
  Objective range  [1e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1.800000e+10
Presolve removed 587 rows and 467 columns
Presolve time: 2.20s
Presolved: 10124 rows, 10238 columns, 1200114 n

In [156]:
np.savez("./Results10p/"+ResultsFileName + ".npz", TotalNumTurbines=Nt,MaxNumTurbines=NuWiWaOcean,SaveFeasibility=SaveFeasibility,
         Save_LCOETarget=Save_LCOETarget, Save_S=Save_S, Save_LCOE_Achieved=Save_LCOE_Achieved, SaveYWind=SaveYWind,SaveYWave=SaveYWave,
         SaveYOcean=SaveYOcean, SaveYWind_LatLong=SaveYWind_LatLong, SaveYWave_LatLong=SaveYWave_LatLong, SaveYOcean_LatLong=SaveYOcean_LatLong,
         SaveCVaR=SaveCVaR)


/home/victor-duraes/anaconda3/envs/Gurobi_py37/lib/python3.7/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
